# Transfer Learning

**Transfer Learning:**

A research problem in machine learning that focuses on storing knowledge gained while solving one problem and applying it to a different but related problem. 

Can leverage an existing neural network architecture proven to work on problems similar to our own

Can leverage a working network architecture which has already learned patterns on similar data to our own (often results in great results with less data)

**Transfer learning use cases:**
* Computer Vision
  - [Imagenet](https://www.image-net.org/): image database organized according to the WordNet hierarchy in which each node of the hierarchy is depicted by hundreds and thousands of images.
  - Currently the best architecture is called EfficientNet
* Natural Language Processing:
  - A subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data. 

## Feature Extraction

In [10]:
import tensorflow as tf
physical_device = tf.config.experimental.list_physical_devices('GPU')
print(f'Device found : {physical_device}') 

Device found : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [1]:
# Get Data (10% of 10 food classes from Food101)
import zipfile

# Download data
!wget -nc -P ../Downloads/ https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip
zip_ref = zipfile.ZipFile('../Downloads/10_food_classes_10_percent.zip')
zip_ref.extractall(path='../Downloads/')
zip_ref.close()


--2022-08-03 11:54:06--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.40.240, 142.251.41.16, 172.217.165.144, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.40.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘../Downloads/10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  36.9MB/s    in 4.4s    

2022-08-03 11:54:11 (36.3 MB/s) - ‘../Downloads/10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [2]:
import os

# Walk through 10 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk('../Downloads/10_food_classes_10_percent'):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in '../Downloads/10_food_classes_10_percent'.
There are 10 directories and 0 images in '../Downloads/10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '../Downloads/10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '../Downloads/10_food_classes_10_percent/test/chicken_curry'.
There are 0 directories and 250 images in '../Downloads/10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '../Downloads/10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '../Downloads/10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '../Downloads/10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '../Downloads/10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '../Downloads/10_food_classes_10_percent/test/pizza'.
There are 0 directories and

### Create data loaders (preparing the data using `ImageDataGenerator`)

In [3]:
# Setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = '../Downloads/10_food_classes_10_percent/train/'
test_dir = '../Downloads/10_food_classes_10_percent/test/'

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print('Training images:')
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                                          target_size=IMAGE_SHAPE,
                                                          batch_size=BATCH_SIZE,
                                                          class_mode='categorical')

print('Testing images:')
test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size=IMAGE_SHAPE,
                                             batch_size=BATCH_SIZE,
                                             class_mode='categorical')

2022-08-03 11:55:00.214224: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Training images:
Found 750 images belonging to 10 classes.
Testing images:
Found 2500 images belonging to 10 classes.


### Setting up callbacks

callbacks are extra functionality you can add to your models to be performed during or after training.  Some of the most popular callbacks:
* Tracking experiments with the `TensorBoard()` callback
  - Log the performance of multiple models and then view and compare these models in a visual way on TensorBoard (a dashboard for inspecting neural network parameters).  Helpful to compare the results of different models on your data.
* Model checkpoint with the `ModelCheckpoint()` callback
  - Save your model as it trains so you can stop training if needed and come back to continue where you left off.  Helpful if training takes a long time and can't be done in one sitting.
* Stopping a model from training (before it trains too long and overfits) with the `EarlyStopping()` callback
  - Leave your model training for an arbitrary amount of time and have it stop training automaticaly when it ceases to improve.  Helpful when you've got a large dataset and don't know how long training will take.

Can be accessed via `tf.keras.callbacks`

In [18]:
# Create a TensorBoard callback (functionized because we need to create a new one for each model)
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=True)
  print(f'Saving TensorBoard logs to {log_dir}')
  return tensorboard_callback

### Creating models using TensorFlow Hub

TensorFlow Hub is a repo of trained ml models

we can access pretrained models on: https://tfhub.dev/
